In [1]:
using BoundaryValueDiffEq, ModelingToolkit, Plots
using ModelingToolkit: t_nounits as t, D_nounits as D

In [ ]:
@constants begin
    # mode numbers of interest
    m = 2
    n = 1

    # device parameters
    R0 = 1

    # poloidal wave numbers 
    ky = 0.25
    km = (m - 1) * ky
    k = (m) * ky
    kp = (m + 1) * ky

    # toroidal wave number
    kz = 1 / R0
    kn = n * kz

    # combined wave numbers
    Km2 = km^2 + kn^2
    K2 = k^2 + kn^2
    Kp2 = kp^2 + kn^2

    # Zeta-components
    ζz = 1
    ζy = 1

    # Lundquist number
    S = 100

    # Alfvénic Mach number
    M = S^(-1 / 2)

    γ = 0.123
end

@variables begin
    ψ(t)
    φ(t)
    f(t)
    #γ(t)
    #x(t)
end
eqs = [
    f ~ tanh(t),
    # m mode equations
    γ * (D(D(φ)) - K2 * φ) ~ -k * f * (D(D(ψ)) - K2 * ψ - (D(D(f)) / f) * ψ),
    γ * ψ - k * f * φ - (1 / S) * (D(D(ψ)) - K2 * ψ) ~ 0,
];

In [7]:
@mtkcompile bvp = System(
    eqs, t
)

Model bvp:
Equations (4):
  4 standard: see equations(bvp)
Unknowns (4): see unknowns(bvp)
  ψ(t)
  ψˍt(t)
  φ(t)
  φˍt(t)
Parameters (10): see parameters(bvp)
  kz [defaults to 1 / R0]
  K2 [defaults to k^2 + kn^2]
  kn [defaults to kz*n]
  ky [defaults to 0.25]
  k [defaults to ky*m]
  γ [defaults to 0.123]
  ⋮
Observed (3): see observed(bvp)

In [ ]:
function fundamental!(du, u, p, t)
    ψ, ψ1, ϕ, ϕ1, x, γ = u
    du[1] = ψ1
    du[2] = (K2 + S * γ) * ψ - S * k * f(t) * ϕ
    du[3] = ϕ1
    du[4] = (-1 / γ) * (-K2 * γ * ϕ + (2*sech(t)^2*ψ - K2*ψ + K2*k*f(t)*ψ + S*γ*k*f(t)*ψ - S*K2*f(t)^2*ϕ))
    du[5] = abs2(ψ)
    du[6] = 0
end